# TVAR PINN: Lag Embedding and 3D Phase Portrait
This notebook expands upon the physics-informed neural network (PINN) demonstration for a time-varying autoregressive
model with constant sliding-window power.  In addition to simulating and training the PINN, it computes a lag embedding
of the observed and predicted signals and displays a 3D phase portrait.  The code is organised into separate sections
for simulation, PINN definition, training, estimation, and analysis so that the PINN can be reused on real data.


## Background
Time-varying autoregressive (TVAR) models describe signals whose autoregressive coefficients change slowly over time.
Estimating these coefficients is challenging: multiple coefficient trajectories can reproduce a signal with similar
energy, so the problem is often ill-posed【988150528581547†L918-L933】.  A physics-informed neural network embeds structural
constraints—here, constant sliding-window power and smooth coefficient variations—into the loss function to improve
identifiability【672647501557241†L152-L160】.  This notebook simulates a TVAR process with low-frequency oscillatory dynamics
similar to local field potentials (LFPs), trains a simple PINN to estimate the time-varying coefficients, and then
reconstructs the predicted signal.  Finally, it computes a delay embedding (lag embedding) and shows the 3D phase
portrait of both the observed and predicted signals.


## Simulation
The following function simulates a TVAR process of order `p` with time-dependent coefficients.
The signal is scaled within a sliding window of length `W` so that its average squared amplitude (power)
remains close to a target value `P0`.  Gaussian noise adds variability.  The function returns the scaled
signal and the true time-varying coefficients as arrays.


In [ ]:

import numpy as np

def simulate_tvar_constant_power(N: int, p: int, W: int, P0: float,
                                coeff_funcs, noise_std: float = 0.1,
                                seed: int = 0, c_min: float | None = None,
                                c_max: float | None = None):
    # Generate a length-N TVAR(p) sequence with constant sliding-window power
    rng = np.random.default_rng(seed)
    x_raw = np.zeros(N)
    x_scaled = np.zeros(N)
    coeffs_true = np.zeros((N, p))
    for t in range(N):
        coeffs_true[t] = [func(t) for func in coeff_funcs]
    S_prev = 0.0
    for t in range(N):
        val = rng.normal(scale=noise_std)
        for k in range(1, p + 1):
            if t >= k:
                val += coeffs_true[t, k - 1] * x_scaled[t - k]
        x_raw[t] = val
        if t == 0:
            S_prev = 0.0
        else:
            if t < W:
                S_prev = np.sum(x_scaled[:t] ** 2)
            else:
                S_prev = S_prev + x_scaled[t - 1] ** 2 - x_scaled[t - W] ** 2
        if t >= W - 1:
            target_energy = P0 * W
            denom = x_raw[t] ** 2 + 1e-12
            c_sq = max((target_energy - S_prev) / denom, 0.0)
            c = np.sqrt(c_sq)
        else:
            c = 1.0
        if c_min is not None or c_max is not None:
            if c_min is None:
                c_min = -np.inf
            if c_max is None:
                c_max = np.inf
            c = np.clip(c, c_min, c_max)
        x_scaled[t] = c * x_raw[t]
    return x_scaled, coeffs_true


## PINN Definition and Training
To estimate the time-varying AR coefficients, we use a simple two-hidden-layer neural network implemented
with NumPy.  The network takes a normalised time input `t_norm` and outputs the coefficients at that time.
The training function minimises a composite loss consisting of an autoregressive residual, an energy penalty
that enforces constant sliding-window power of the reconstructed signal, and a smoothness penalty on
coefficient variations.  These functions are modular so they can be reused for real data by replacing the
simulated signal `x` with a real time series.


In [ ]:

import numpy as np

class SimplePINN:
    def __init__(self, p: int, h1: int = 20, h2: int = 20, seed: int = 0):
        rng = np.random.default_rng(seed)
        self.W1 = rng.normal(scale=0.5, size=(1, h1))
        self.b1 = np.zeros(h1)
        self.W2 = rng.normal(scale=0.5, size=(h1, h2))
        self.b2 = np.zeros(h2)
        self.W3 = rng.normal(scale=0.5, size=(h2, p))
        self.b3 = np.zeros(p)
    @staticmethod
    def tanh(x):
        return np.tanh(x)
    @staticmethod
    def dtanh(x):
        return 1.0 - np.tanh(x) ** 2
    def forward(self, t: np.ndarray):
        z1 = t @ self.W1 + self.b1
        h1 = self.tanh(z1)
        z2 = h1 @ self.W2 + self.b2
        h2 = self.tanh(z2)
        out = h2 @ self.W3 + self.b3
        cache = {"t": t, "z1": z1, "h1": h1, "z2": z2, "h2": h2}
        return out, cache
    def backward(self, cache: dict, dL_dout: np.ndarray):
        z1, h1 = cache["z1"], cache["h1"]
        z2, h2 = cache["z2"], cache["h2"]
        t = cache["t"]
        M = dL_dout.shape[0]
        dW3 = np.zeros_like(self.W3)
        db3 = np.zeros_like(self.b3)
        dW2 = np.zeros_like(self.W2)
        db2 = np.zeros_like(self.b2)
        dW1 = np.zeros_like(self.W1)
        db1 = np.zeros_like(self.b1)
        for i in range(M):
            g_c = dL_dout[i]
            h2_i = h2[i]
            dW3 += np.outer(h2_i, g_c)
            db3 += g_c
            g_h2 = g_c @ self.W3.T
            g_z2 = g_h2 * self.dtanh(z2[i])
            h1_i = h1[i]
            dW2 += np.outer(h1_i, g_z2)
            db2 += g_z2
            g_h1 = g_z2 @ self.W2.T
            g_z1 = g_h1 * self.dtanh(z1[i])
            t_i = t[i, 0]
            dW1 += t_i * g_z1.reshape(1, -1)
            db1 += g_z1
        grads = {"W1": dW1, "b1": db1, "W2": dW2, "b2": db2, "W3": dW3, "b3": db3}
        return grads
    def step(self, grads: dict, lr: float):
        self.W1 -= lr * grads["W1"]
        self.b1 -= lr * grads["b1"]
        self.W2 -= lr * grads["W2"]
        self.b2 -= lr * grads["b2"]
        self.W3 -= lr * grads["W3"]
        self.b3 -= lr * grads["b3"]


def train_pinn(x: np.ndarray, p: int, W: int, P0: float, n_epochs: int = 200,
               lr: float = 0.01, lambda_energy: float = 0.1, lambda_smooth: float = 0.01,
               h1: int = 20, h2: int = 20, seed: int = 0, scale_coeff: float = 0.5):
    N = len(x)
    t_vals = np.linspace(0.0, 1.0, N)
    train_t = t_vals.reshape(-1, 1)
    pinn = SimplePINN(p=p, h1=h1, h2=h2, seed=seed)
    hist_total, hist_ar, hist_energy, hist_smooth = [], [], [], []
    for epoch in range(n_epochs):
        out, cache = pinn.forward(train_t)
        coeffs_out = scale_coeff * out
        residuals = np.zeros(N)
        x_hat = np.zeros(N)
        for i in range(p, N):
            past_vals = np.array([x[i - k] for k in range(1, p + 1)])
            coeffs = coeffs_out[i]
            x_hat[i] = np.dot(coeffs, past_vals)
            residuals[i] = x[i] - x_hat[i]
        loss_ar = np.mean(residuals[p:] ** 2)
        P_hat = np.zeros(N)
        for i in range(p, N):
            start = max(p, i - W + 1)
            window = x_hat[start:i + 1]
            P_hat[i] = np.mean(window ** 2) if window.size > 0 else 0.0
        loss_energy = np.mean((P_hat[p:] - P0) ** 2)
        diff = coeffs_out[1:] - coeffs_out[:-1]
        loss_smooth = np.mean(np.sum(diff ** 2, axis=1))
        loss_total = loss_ar + lambda_energy * loss_energy + lambda_smooth * loss_smooth
        hist_total.append(loss_total)
        hist_ar.append(loss_ar)
        hist_energy.append(loss_energy)
        hist_smooth.append(loss_smooth)
        dL_dout = np.zeros_like(out)
        for i in range(p, N):
            past_vals = np.array([x[i - k] for k in range(1, p + 1)])
            d = -2.0 * residuals[i] / (N - p)
            dL_dout[i] += d * (scale_coeff * past_vals)
        dL_dPhat = (2.0 / (N - p)) * (P_hat - P0)
        dL_d_xhat = np.zeros(N)
        for j in range(p, N):
            i_start = j
            i_end = min(N - 1, j + W - 1)
            if i_end >= i_start:
                s = np.sum(dL_dPhat[i_start:i_end + 1])
                dL_d_xhat[j] = s * (2.0 / W) * x_hat[j]
        for j in range(p, N):
            past_vals = np.array([x[j - k] for k in range(1, p + 1)])
            dL_dout[j] += lambda_energy * dL_d_xhat[j] * (scale_coeff * past_vals)
        for k in range(1, N):
            grad = np.zeros(p)
            if k < N - 1:
                grad += 2 * (coeffs_out[k] - coeffs_out[k + 1])
            if k > 0:
                grad += 2 * (coeffs_out[k] - coeffs_out[k - 1])
            dL_dout[k] += (lambda_smooth / N) * grad
        dL_dout *= scale_coeff
        grads = pinn.backward(cache, dL_dout)
        pinn.step(grads, lr)
    history = {"total": hist_total, "ar": hist_ar, "energy": hist_energy, "smooth": hist_smooth}
    return pinn, history


## Estimation and Analysis
After training the PINN, we estimate the time-varying coefficients and reconstruct the predicted signal using the
observed past samples.  We compute the delay (tau) for phase-space reconstruction by finding the first local minimum
of the autocorrelation function of the observed signal.  The embedding dimension for the 3D phase portrait is set
to 3.  Finally, we compute and plot the 3D phase portrait for the observed and predicted signals.


In [ ]:

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.signal import argrelextrema

# Parameters for simulation
N = 600
p_order = 2
W = 40
P0 = 0.5
freq1 = 1/150
freq2 = 1/200

# Define coefficient functions
coeff_funcs = [lambda t: 0.6 + 0.35 * np.sin(2 * np.pi * (1/150) * t),
               lambda t: -0.5 + 0.25 * np.cos(2 * np.pi * (1/200) * t)]

# Simulate the signal and true coefficients
x, coeffs_true = simulate_tvar_constant_power(N, p_order, W, P0, coeff_funcs,
                                             noise_std=0.1, seed=42, c_max=5.0)

# Train the PINN
pinn_model, history = train_pinn(x, p_order, W, P0, n_epochs=200, lr=0.01,
                                 lambda_energy=0.1, lambda_smooth=0.01,
                                 h1=20, h2=20, seed=1, scale_coeff=0.5)

# Predict coefficients for all time points
norm_t = np.linspace(0.0, 1.0, N).reshape(-1, 1)
coeff_pred_raw, _ = pinn_model.forward(norm_t)
scale_coeff = 0.5
coeff_pred = scale_coeff * coeff_pred_raw

# Reconstruct predicted signal using predicted coefficients and observed past samples
pred_signal = np.zeros(N)
pred_signal[:p_order] = x[:p_order]
for i in range(p_order, N):
    coeffs = coeff_pred[i]
    pred_signal[i] = coeffs[0] * x[i-1] + coeffs[1] * x[i-2]

# Compute autocorrelation of the observed signal to estimate tau
max_lag = 100
acf = np.array([np.corrcoef(x[:-lag], x[lag:])[0,1] if lag > 0 else 1.0 for lag in range(max_lag)])
mins = argrelextrema(acf, np.less)[0]
tau = int(mins[0]) if len(mins) > 0 else 1

embedding_dim = 3
# Compute lag embedding for observed and predicted signals
max_index = N - (embedding_dim - 1) * tau
embedding_actual = np.column_stack([x[i: i + max_index] for i in range(0, embedding_dim * tau, tau)])
embedding_pred = np.column_stack([pred_signal[i: i + max_index] for i in range(0, embedding_dim * tau, tau)])

# Print tau and embedding dimension
print(f"Estimated time delay (tau): {tau}")
print(f"Embedding dimension (p): {embedding_dim}")

# Plot the training loss components
plt.figure(figsize=(8, 3))
plt.plot(history['total'], label='Total loss')
plt.plot(history['ar'], label='AR residual')
plt.plot(history['energy'], label='Energy penalty')
plt.plot(history['smooth'], label='Smoothness penalty')
plt.yscale('log')
plt.xlabel('Epoch')
plt.ylabel('Loss (log scale)')
plt.title('Loss components during training')
plt.legend()
plt.tight_layout()
plt.show()

# Plot true vs predicted coefficients
plt.figure(figsize=(8, 4))
plt.plot(coeffs_true[:, 0], label='True a1')
plt.plot(coeff_pred[:, 0], label='Predicted a1', linestyle='--')
plt.xlabel('Time index')
plt.ylabel('Coefficient a1')
plt.title('First AR coefficient')
plt.legend()
plt.tight_layout()
plt.show()

plt.figure(figsize=(8, 4))
plt.plot(coeffs_true[:, 1], label='True a2')
plt.plot(coeff_pred[:, 1], label='Predicted a2', linestyle='--')
plt.xlabel('Time index')
plt.ylabel('Coefficient a2')
plt.title('Second AR coefficient')
plt.legend()
plt.tight_layout()
plt.show()

# Plot 3D phase portrait
fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111, projection='3d')
# Subsample for clarity
idx = np.arange(0, embedding_actual.shape[0], 5)
ax.plot(embedding_actual[idx, 0], embedding_actual[idx, 1], embedding_actual[idx, 2], label='Observed', alpha=0.7)
ax.plot(embedding_pred[idx, 0], embedding_pred[idx, 1], embedding_pred[idx, 2], label='Predicted', alpha=0.7)
ax.set_title(f'3D Phase Portrait (tau = {tau})')
ax.set_xlabel('x(t)')
ax.set_ylabel(f'x(t+{tau})')
ax.set_zlabel(f'x(t+2*{tau})')
ax.legend()
plt.tight_layout()
plt.show()


## Conclusion
This notebook demonstrated a complete workflow for simulating, training, and analysing a time-varying autoregressive
process using a physics-informed neural network.  The code is organised into modular functions so that the PINN
and the lag-embedding analysis can be applied to real data by replacing the simulated signal with an observed time
series.  The estimated time delay (tau) and embedding dimension (p) provide the parameters needed to reconstruct
a phase space in three dimensions and visualise the dynamical structure of the signal.
